# `specfile_example()` - Output scan(s) to a SPEC data file.

One of the common concerns is how to access data from bluesky's database.  The standard way is to replay the document stream from each of the scans through a bluesky callback that writes the data to the desired file format.  Here, we write data to the SPEC file format.

First, we must load the libraries we'll need.

In [1]:
from databroker import Broker
from APS_BlueSky_tools.demo import specfile_example

Next, we create an instance of the *Broker()* using our mongodb database. (For this to work, we know we already have a file located in `~/.config/databroker/mongodb_config.yml` that describes the databroker configuration for mongodb.)

In [2]:
db = Broker.named("mongodb_config")

# get the most recent scan, by steps

The databroker instance, `db`, provides access to its scans by several means.  One way is to consider `db` as a list and retreive the last item from the list.  This will return a *header* to the scan.  The *header* is the common reference to be used.  As is the common term, we will call it `h` and print its *start* document.

For this first example, we'll work through the steps one by one.

In [3]:
h = db[-1]
h.start

INFO:databroker._core:Interpreting key = -1 as an integer


{'time': 1545892560.0517623,
 'uid': '9bc1fe93-e56b-4dfc-a2bc-ee91b6f88200',
 'hints': {},
 'software_versions': {'python': '3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 17:14:51) \n[GCC 7.2.0]',
  'PyEpics': '3.3.1',
  'bluesky': '1.4.1',
  'ophyd': '1.3.0',
  'databroker': '0.11.3',
  'APS_Bluesky_Tools': '0.0.40+1.g8705698'},
 'iso8601': '2018-12-27 00:36:00.050644',
 'plan_type': 'generator',
 'plan_name': 'snapshot',
 'username': 'mintadmin',
 'scan_id': 1,
 'plan_description': 'archive snapshot of ophyd Signals (usually EPICS PVs)',
 'login_id': 'mintadmin@mint-vm',
 'hostname': 'mint-vm',
 'purpose': 'example'}

The databroker provides a simple table view of this scan (header):

In [4]:
h.table()

,time,signal_0,signal_1
seq_num,,,
1,2018-12-27 00:36:00.193227053,0.500442,21.118952


Let's write it as a SPEC data file (namely: `/tmp/spec1.dat`):

In [5]:
specfile_example(h, filename="/tmp/spec1.dat")

INFO:APS_BlueSky_tools.filewriters:wrote header to SPEC file: /tmp/spec1.dat
INFO:APS_BlueSky_tools.filewriters:wrote scan 1 to SPEC file: /tmp/spec1.dat
INFO:APS_BlueSky_tools.demo:
#S 1  snapshot()
#D Thu Dec 27 00:36:00 2018
#C Thu Dec 27 00:36:00 2018.  plan_type = generator
#C Thu Dec 27 00:36:00 2018.  uid = 9bc1fe93-e56b-4dfc-a2bc-ee91b6f88200
#MD hostname = mint-vm
#MD iso8601 = 2018-12-27 00:36:00.050644
#MD login_id = mintadmin@mint-vm
#MD plan_description = archive snapshot of ophyd Signals (usually EPICS PVs)
#MD purpose = example
#MD software_versions = {'python': '3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 17:14:51) \n[GCC 7.2.0]', 'PyEpics': '3.3.1', 'bluesky': '1.4.1', 'ophyd': '1.3.0', 'databroker': '0.11.3', 'APS_Bluesky_Tools': '0.0.40+1.g8705698'}
#MD username = mintadmin
#N 1
#L Epoch_float  signal_0  signal_1  Epoch
0.1414647102355957 0.5004422061971736 21.11895183692659 0
#C Thu Dec 27 00:36:00 2018.  num_events_primary = 1
#C Thu Dec 27 00:36:00 201

Let's view that file from disk storage:

In [6]:
!cat /tmp/spec1.dat

#F /tmp/spec1.dat
#E 1546470175
#D Wed Jan 02 17:02:55 2019
#C BlueSky  user = mintadmin  host = mint-vm

#S 1  snapshot()
#D Thu Dec 27 00:36:00 2018
#C Thu Dec 27 00:36:00 2018.  plan_type = generator
#C Thu Dec 27 00:36:00 2018.  uid = 9bc1fe93-e56b-4dfc-a2bc-ee91b6f88200
#MD hostname = mint-vm
#MD iso8601 = 2018-12-27 00:36:00.050644
#MD login_id = mintadmin@mint-vm
#MD plan_description = archive snapshot of ophyd Signals (usually EPICS PVs)
#MD purpose = example
#MD software_versions = {'python': '3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 17:14:51) \n[GCC 7.2.0]', 'PyEpics': '3.3.1', 'bluesky': '1.4.1', 'ophyd': '1.3.0', 'databroker': '0.11.3', 'APS_Bluesky_Tools': '0.0.40+1.g8705698'}
#MD username = mintadmin
#N 1
#L Epoch_float  signal_0  signal_1  Epoch
0.1414647102355957 0.5004422061971736 21.11895183692659 0
#C Thu Dec 27 00:36:00 2018.  num_events_primary = 1
#C Thu Dec 27 00:36:00 2018.  exit_status = success


We see that the output of the `specfile_example()` command includes the content of the SPEC file.  For the remaining examples, we'll shortcut the call to the command view its output

# a range of recent scans

We can continue to consider the `db` object as a list and use list slicing to access a range of recent scans.

In [7]:
specfile_example(db[-6:][::-3], filename="spec2.dat")

INFO:databroker._core:Interpreting key = slice(-6, None, None) as a slice
INFO:APS_BlueSky_tools.filewriters:wrote header to SPEC file: spec2.dat
INFO:APS_BlueSky_tools.filewriters:wrote scan 1123 to SPEC file: spec2.dat
INFO:APS_BlueSky_tools.demo:
#S 1123  TuneAxis.multi_pass_tune()
#D Wed Dec 26 22:53:10 2018
#C Wed Dec 26 22:53:10 2018.  plan_type = generator
#C Wed Dec 26 22:53:10 2018.  uid = 9c2d8735-3653-4b42-8428-aed4f2338cbc
#MD APS_BlueSky_tools_VERSION = 0.0.35
#MD BLUESKY_VERSION = 1.4.1
#MD OPHYD_VERSION = 1.3.0
#MD SESSION_START = 2018-12-11 14:03:08.460986
#MD beamline_id = developer
#MD ipython_session_start = 2018-02-14 12:54:06.447450
#MD login_id = mintadmin@mint-vm
#MD md = {'activity': 'TuneAxis development and testing', 'peak_model': 'pseudo Voigt', 'peak_scale': 100000.0, 'peak_center': -1.4910027475019858, 'peak_sigma': 0.035867168639114114, 'peak_eta': 0.274478257770014, 'peak_bkg': 0.009072591644989138}
#MD pass = 1
#MD pass_max = 6
#MD pid = 22941
#MD propos

# a specific scan

The `db` object allows us to access scans by UUID (or any shorter version that remains unique in the database).

In [8]:
specfile_example(db["2da6161b"], filename="spec3.dat")

INFO:databroker._core:Interpreting key = 2da6161b as a str
INFO:APS_BlueSky_tools.filewriters:wrote header to SPEC file: spec3.dat
INFO:APS_BlueSky_tools.filewriters:wrote scan 14 to SPEC file: spec3.dat
INFO:APS_BlueSky_tools.demo:
#S 14  scan(detectors=['synthetic_pseudovoigt'], num=219, motor=['m1'], start=-2, stop=0, per_step=None)
#D Thu Oct 26 11:26:39 2017
#C Thu Oct 26 11:26:39 2017.  plan_type = generator
#C Thu Oct 26 11:26:39 2017.  uid = 2da6161b-bc43-4eb2-b696-ecc1f256a366
#MD EPICS_BASE = /usr/local/epics/base
#MD EPICS_BASE_BIN = /usr/local/epics/base/bin/linux-x86_64
#MD EPICS_BASE_LIB = /usr/local/epics/base/lib/linux-x86_64
#MD EPICS_CA_MAX_ARRAY_BYTES = 16777216
#MD EPICS_EXT = /usr/local/epics/opi
#MD EPICS_EXT_BIN = /usr/local/epics/opi/bin/linux-x86_64
#MD EPICS_EXT_LIB = /usr/local/epics/opi/lib/linux-x86_64
#MD EPICS_HOST_ARCH = linux-x86_64
#MD EPICS_ROOT = /usr/local/epics
#MD beamline_id = developer__YOUR_BEAMLINE_HERE
#MD login_id = mintadmin@mint-vm
#MD mot

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.demo:Look at SPEC data file: spec3.dat


# a list of specific scans, by UID

Suppose we have a list of scans where we know the UID of each one, we can build a list of headers and write a SPEC data file with that list.  Here, we have such a list of tuning scans.

In [9]:
hh = [db[uid] for uid in "9c2d8735 099a1882 98c92dd2 61cfbe3b".split()]
specfile_example(hh, filename="spec_tunes.dat")

INFO:databroker._core:Interpreting key = 9c2d8735 as a str
INFO:databroker._core:Interpreting key = 099a1882 as a str
INFO:databroker._core:Interpreting key = 98c92dd2 as a str
INFO:databroker._core:Interpreting key = 61cfbe3b as a str
INFO:APS_BlueSky_tools.filewriters:wrote header to SPEC file: spec_tunes.dat
INFO:APS_BlueSky_tools.filewriters:wrote scan 1123 to SPEC file: spec_tunes.dat
INFO:APS_BlueSky_tools.demo:
#S 1123  TuneAxis.multi_pass_tune()
#D Wed Dec 26 22:53:10 2018
#C Wed Dec 26 22:53:10 2018.  plan_type = generator
#C Wed Dec 26 22:53:10 2018.  uid = 9c2d8735-3653-4b42-8428-aed4f2338cbc
#MD APS_BlueSky_tools_VERSION = 0.0.35
#MD BLUESKY_VERSION = 1.4.1
#MD OPHYD_VERSION = 1.3.0
#MD SESSION_START = 2018-12-11 14:03:08.460986
#MD beamline_id = developer
#MD ipython_session_start = 2018-02-14 12:54:06.447450
#MD login_id = mintadmin@mint-vm
#MD md = {'activity': 'TuneAxis development and testing', 'peak_model': 'pseudo Voigt', 'peak_scale': 100000.0, 'peak_center': -1.491

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.demo:Look at SPEC data file: spec_tunes.dat


## "scan" plans in 2018-12 until the start of the NYE party

The `db` object allows for filtering arguments based on any keywords in the *start* document and also by time.  Here, we filter between certain dates and also by `plan name`.  Also, we write to the default data file: `test_specdata.txt`.

In [10]:
specfile_example(db(plan_name="scan"))

INFO:APS_BlueSky_tools.filewriters:wrote header to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.filewriters:wrote scan 1094 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 1094  scan(detectors=['scaler'], num=5, args=['m1', -1, 1, 'm2', 0, 1], per_step=None)
#D Wed Dec 05 13:39:39 2018
#C Wed Dec 05 13:39:39 2018.  plan_type = generator
#C Wed Dec 05 13:39:39 2018.  uid = e6a293cf-e517-4a11-8c76-81b6286e32db
#MD APS_BlueSky_tools_VERSION = 0.0.35
#MD BLUESKY_VERSION = 1.4.1
#MD OPHYD_VERSION = 1.3.0
#MD SESSION_START = 2018-12-05 13:37:25.804493
#MD beamline_id = developer
#MD ipython_session_start = 2018-02-14 12:54:06.447450
#MD login_id = mintadmin@mint-vm
#MD motors = ['m1', 'm2']
#MD num_intervals = 4
#MD num_points = 5
#MD pid = 4309
#MD plan_pattern = inner_product
#MD plan_pattern_args = {'num': 5, 'args': ["EpicsMotor(prefix='prj:m1', name='m1', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint'], configuration_attrs=['user_offse

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 582 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 582  scan(detectors=['scaler'], num=5, motor=['m1'], start=-5, stop=0, per_step=None)
#D Mon Dec 18 17:27:12 2017
#C Mon Dec 18 17:27:12 2017.  plan_type = generator
#C Mon Dec 18 17:27:12 2017.  uid = 1242c172-a3bd-44b7-a722-c9610343aad1
#MD beamline_id = developer
#MD login_id = mintadmin@mint-vm
#MD md = {'demo': 'MONA motor scan', 'purpose': 'development'}
#MD motors = ['m1']
#MD num_intervals = 4
#MD num_points = 5
#MD pid = 18830
#MD plan_pattern = linspace
#MD plan_pattern_args = {'start': -5, 'stop': 0, 'num': 5}
#MD plan_pattern_module = numpy
#MD proposal_id = None
#N 5
#L m1_user_setpoint  scaler_channels_chan1  scaler_channels_chan2  scaler_channels_chan3  scaler_channels_chan6  scaler_time  Epoch_float  Epoch  m1
-5.0 6000000.0 2.0 2.0 4.0 0.6 3.482491970062256 3 -5.0
-3.75

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 576 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 576  scan(detectors=['scaler'], num=10, motor=['m1'], start=-5, stop=0, per_step=None)
#D Mon Dec 18 17:16:31 2017
#C Mon Dec 18 17:16:31 2017.  plan_type = generator
#C Mon Dec 18 17:16:31 2017.  uid = 60197d82-e3e2-4959-9edf-fb33adafa8b0
#MD beamline_id = developer
#MD login_id = mintadmin@mint-vm
#MD md = {'demo': 'MONA motor scan', 'purpose': 'development'}
#MD motors = ['m1']
#MD num_intervals = 9
#MD num_points = 10
#MD pid = 17631
#MD plan_pattern = linspace
#MD plan_pattern_args = {'start': -5, 'stop': 0, 'num': 10}
#MD plan_pattern_module = numpy
#MD proposal_id = None
#N 10
#L m1_user_setpoint  scaler_channels_chan1  scaler_channels_chan2  scaler_channels_chan3  scaler_channels_chan6  scaler_time  Epoch_float  Epoch  m1
-5.0 6000000.0 2.0 3.0 3.0 0.6 1.0546956062316895 1 -5.0


INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 571 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 571  scan(detectors=['scaler'], num=3, motor=['m1'], start=-5, stop=0, per_step=None)
#D Mon Dec 18 16:19:46 2017
#C Mon Dec 18 16:19:46 2017.  plan_type = generator
#C Mon Dec 18 16:19:46 2017.  uid = a7cdf915-1477-4a11-a73a-3f68aae2b575
#MD beamline_id = developer
#MD login_id = mintadmin@mint-vm
#MD md = {'demo': 'MONA motor scan', 'purpose': 'development'}
#MD motors = ['m1']
#MD num_intervals = 2
#MD num_points = 3
#MD pid = 15376
#MD plan_pattern = linspace
#MD plan_pattern_args = {'start': -5, 'stop': 0, 'num': 3}
#MD plan_pattern_module = numpy
#MD proposal_id = None
#N 3
#L m1_user_setpoint  scaler_channels_chan1  scaler_channels_chan2  scaler_channels_chan3  scaler_channels_chan6  scaler_time  Epoch_float  Epoch  m1
-5.0 6000000.0 1.0 4.0 2.0 0.6 5.976656198501587 6 -5.0
-2.5 

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 564 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 564  scan(detectors=['scaler'], num=3, motor=['m1'], start=-1, stop=0, per_step=None)
#D Mon Dec 18 16:06:20 2017
#C Mon Dec 18 16:06:20 2017.  plan_type = generator
#C Mon Dec 18 16:06:20 2017.  uid = b38cde34-0869-47b8-bbc6-6ba81d4b9824
#MD beamline_id = developer
#MD login_id = mintadmin@mint-vm
#MD md = {'demo': 'MONA motor scan', 'purpose': 'development'}
#MD motors = ['m1']
#MD num_intervals = 2
#MD num_points = 3
#MD pid = 14189
#MD plan_pattern = linspace
#MD plan_pattern_args = {'start': -1, 'stop': 0, 'num': 3}
#MD plan_pattern_module = numpy
#MD proposal_id = None
#N 3
#L m1_user_setpoint  scaler_channels_chan1  scaler_channels_chan2  scaler_channels_chan3  scaler_channels_chan6  scaler_time  Epoch_float  Epoch  m1
-1.0 6000000.0 1.0 2.0 2.0 0.6 2.04184889793396 2 -1.0
-0.5 6

...

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 4 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 4  scan(detectors=['noisy'], num=219, motor=['m1'], start=-1.5, stop=-0.5, per_step=None)
#D Thu Oct 26 11:22:22 2017
#C Thu Oct 26 11:22:22 2017.  plan_type = generator
#C Thu Oct 26 11:22:22 2017.  uid = f24bf2cc-ea1a-4a73-a67e-c6767a9ee518
#MD EPICS_BASE = /usr/local/epics/base
#MD EPICS_BASE_BIN = /usr/local/epics/base/bin/linux-x86_64
#MD EPICS_BASE_LIB = /usr/local/epics/base/lib/linux-x86_64
#MD EPICS_CA_MAX_ARRAY_BYTES = 16777216
#MD EPICS_EXT = /usr/local/epics/opi
#MD EPICS_EXT_BIN = /usr/local/epics/opi/bin/linux-x86_64
#MD EPICS_EXT_LIB = /usr/local/epics/opi/lib/linux-x86_64
#MD EPICS_HOST_ARCH = linux-x86_64
#MD EPICS_ROOT = /usr/local/epics
#MD beamline_id = developer__YOUR_BEAMLINE_HERE
#MD login_id = mintadmin@mint-vm
#MD motors = ['m1']
#MD num_intervals = 218
#MD num_po

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 3 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 3  scan(detectors=['noisy'], num=219, motor=['m1'], start=-1.5, stop=-0.5, per_step=None)
#D Thu Oct 26 11:22:08 2017
#C Thu Oct 26 11:22:08 2017.  plan_type = generator
#C Thu Oct 26 11:22:08 2017.  uid = 3953e8e0-038b-41a0-aab6-95ada41b8e6b
#MD EPICS_BASE = /usr/local/epics/base
#MD EPICS_BASE_BIN = /usr/local/epics/base/bin/linux-x86_64
#MD EPICS_BASE_LIB = /usr/local/epics/base/lib/linux-x86_64
#MD EPICS_CA_MAX_ARRAY_BYTES = 16777216
#MD EPICS_EXT = /usr/local/epics/opi
#MD EPICS_EXT_BIN = /usr/local/epics/opi/bin/linux-x86_64
#MD EPICS_EXT_LIB = /usr/local/epics/opi/lib/linux-x86_64
#MD EPICS_HOST_ARCH = linux-x86_64
#MD EPICS_ROOT = /usr/local/epics
#MD beamline_id = developer__YOUR_BEAMLINE_HERE
#MD login_id = mintadmin@mint-vm
#MD motors = ['m1']
#MD num_intervals = 218
#MD num_po

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 2 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 2  scan(detectors=['noisy'], num=219, motor=['m1'], start=-1.5, stop=-0.5, per_step=None)
#D Thu Oct 26 11:21:42 2017
#C Thu Oct 26 11:21:42 2017.  plan_type = generator
#C Thu Oct 26 11:21:42 2017.  uid = 25b4c903-e9ad-435c-ae9e-b2ff5682e30e
#MD EPICS_BASE = /usr/local/epics/base
#MD EPICS_BASE_BIN = /usr/local/epics/base/bin/linux-x86_64
#MD EPICS_BASE_LIB = /usr/local/epics/base/lib/linux-x86_64
#MD EPICS_CA_MAX_ARRAY_BYTES = 16777216
#MD EPICS_EXT = /usr/local/epics/opi
#MD EPICS_EXT_BIN = /usr/local/epics/opi/bin/linux-x86_64
#MD EPICS_EXT_LIB = /usr/local/epics/opi/lib/linux-x86_64
#MD EPICS_HOST_ARCH = linux-x86_64
#MD EPICS_ROOT = /usr/local/epics
#MD beamline_id = developer__YOUR_BEAMLINE_HERE
#MD login_id = mintadmin@mint-vm
#MD motors = ['m1']
#MD num_intervals = 218
#MD num_po

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.filewriters:wrote scan 1 to SPEC file: test_specdata.txt
INFO:APS_BlueSky_tools.demo:
#S 1  scan(detectors=['noisy'], num=219, motor=['m1'], start=-1.5, stop=-0.5, per_step=None)
#D Thu Oct 26 11:21:28 2017
#C Thu Oct 26 11:21:28 2017.  plan_type = generator
#C Thu Oct 26 11:21:28 2017.  uid = 3fe59011-a3b2-4ac4-a0fd-5a5568dc13a3
#MD EPICS_BASE = /usr/local/epics/base
#MD EPICS_BASE_BIN = /usr/local/epics/base/bin/linux-x86_64
#MD EPICS_BASE_LIB = /usr/local/epics/base/lib/linux-x86_64
#MD EPICS_CA_MAX_ARRAY_BYTES = 16777216
#MD EPICS_EXT = /usr/local/epics/opi
#MD EPICS_EXT_BIN = /usr/local/epics/opi/bin/linux-x86_64
#MD EPICS_EXT_LIB = /usr/local/epics/opi/lib/linux-x86_64
#MD EPICS_HOST_ARCH = linux-x86_64
#MD EPICS_ROOT = /usr/local/epics
#MD beamline_id = developer__YOUR_BEAMLINE_HERE
#MD login_id = mintadmin@mint-vm
#MD motors = ['m1']
#MD num_intervals = 218
#MD num_po

INFO:APS_BlueSky_tools.demo:############################################################
INFO:APS_BlueSky_tools.demo:Look at SPEC data file: test_specdata.txt
